In [1]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=5)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 569, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 569 (delta 183), reused 184 (delta 107), pack-reused 266
Receiving objects: 100% (569/569), 49.04 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (282/282), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.26.1
Using datasets v2.10.1
Using accelerate v0.16.0
Using sentencepiece v0.1.97


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

디코딩 메서드 직접 구현.  

'Transformers are the'를 입력 프롬프트로 사용하고 여덟 번의 타임스텝 동안 디코딩을 수행한다. 각 타임스텝에서 프롬프트의 마지막 토큰에 대한 로짓을 선택하고 소프트맥스를 적용해 확률 분포를 얻는다. 그 다음 확률이 가장 높은 토큰을 다음 토큰으로 선택하고 입력 시퀀스에 추가한 후에 이 과정을 다시 반복한다. 또 대안을 시각적으로 보여주기 위해 타임스텝마다 확률이 가장 높은 토큰을 다섯 개 저장한다.

In [3]:
input_txt = "transformers are the"

input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
iteration = dict()
iteration['Input'] = tokenizer.decode(input_ids[0])
output = model(input_ids = input_ids)

In [6]:
tokenizer.tokenize(input_txt)

['transform', 'ers', 'Ġare', 'Ġthe']

In [4]:
iteration

{'Input': 'transformers are the'}

In [5]:
output.logits[0, -1, :]

tensor([ -98.5361,  -97.8216, -101.2157,  ...,  -99.9019, -102.0636,
         -96.9101], device='cuda:0', grad_fn=<SliceBackward0>)

In [6]:
import pandas as pd

input_txt = "transformers are the"
input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
iterations = []
n_steps = 8 #8번의 타임스텝동안 디코딩 수행
choices_per_step = 5

with torch.no_grad():
  for _ in range(n_steps): #각 타임스텝에서
    iteration = dict()
    iteration['Input'] = tokenizer.decode(input_ids[0])
    output = model(input_ids = input_ids)
    #첫 번째 배치의 마지막 토큰 로짓을 선택해 소프트맥스를 적용
    next_token_logits = output.logits[0, -1, :] #첫 번째 배치의 마지막 토큰 로짓 선택.  output.logits[0, -1, :] = [-98.5361, -97.8216, ... , -96.9101], 50257개
    next_token_probs = torch.softmax(next_token_logits, dim = -1) #소프트맥스 적용 -> 확률로 변환
    sorted_ids = torch.argsort(next_token_probs, dim=-1, descending = True) #정렬 인덱스 변환(내림차순)
    #가장 높은 확률의 토큰을 저장한다.  
    for choice_idx in range(choices_per_step): #시각적으로 보여주기 위함. 타임스텝마다 확률이 가장 높은 토큰을 5개 저장
      token_id = sorted_ids[choice_idx] #가장 큰 값의 인덱스부터 차례대로
      token_prob = next_token_probs[token_id].cpu().numpy()
      token_choice = (
          f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
      )
      iteration[f"Choice {choice_idx + 1}"] = token_choice
    #예측한 다음 토큰을 입력에 추가한다.
    input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim = -1)
    iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,transformers are the,same (9.33%),most (6.50%),only (2.87%),ones (2.77%),best (2.26%)
1,transformers are the same,as (35.71%),. (12.00%),", (8.19%)",for (3.61%),thing (2.24%)
2,transformers are the same as,the (14.87%),those (11.36%),in (10.22%),they (4.01%),for (3.90%)
3,transformers are the same as the,ones (15.79%),original (3.46%),one (2.39%),default (2.16%),other (1.69%)
4,transformers are the same as the ones,used (21.92%),in (12.15%),that (7.46%),we (6.38%),you (5.55%)
5,transformers are the same as the ones used,in (36.79%),for (21.14%),by (18.54%),to (8.15%),on (5.71%)
6,transformers are the same as the ones used in,the (22.25%),other (1.40%),a (1.34%),Java (1.29%),Windows (1.03%)
7,transformers are the same as the ones used in the,original (6.24%),previous (4.81%),game (1.49%),first (0.90%),current (0.87%)


예측하는 데 한 번의 정방향 패스로 충분한 시퀀스 분류 등의 작업과 달리, 텍스트 생성은 한 번에 하나의 출력 토큰을 디코딩한다.  

그리드 서치 구현은 어렵지 않지만, 더 복잡한 디코딩 방법을 알아보기 위해 트랜스포머스에 내장된 generate() 함수를 사용해보자. 앞선 결과를 재현하기 위해 샘플링을 끄고 생성 토큰의 개수를 max_new_tokens 매개변수로 지정한다.

In [7]:
input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
output = model.generate(input_ids, max_new_tokens = n_steps, do_sample = False)
print(tokenizer.decode(output[0]))

transformers are the same as the ones used in the original


다른 시도를 해보자. OpenAI의 유니콘 기사를 재현해보겠다.

앞에서처럼 토크나이저로 프롬프트를 인코딩하고 긴 텍스트 시퀀스를 생성하기 위해 max_length에 큰 값을 지정하겠다.

In [8]:
max_length = 128
input_txt =  """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors = 'pt')['input_ids'].to(device)
output_greedy = model.generate(input_ids, max_length = max_length, do_sample = False)
print(tokenizer.decode(output_greedy[0]))
output_greedy

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


"The unicorns were very intelligent, and they were very intelligent," said Dr.
David S. Siegel, a professor of anthropology at the University of California,
Berkeley. "They were very intelligent, and they were very intelligent, and they
were very intelligent, and they were very intelligent, and they were very
intelligent, and they were very intelligent, and they were very intelligent, and
they were very


tensor([[  818,   257, 14702,  4917,    11, 11444,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13,   628,   198,     1,   464, 28000,
         19942,   547,   845, 12661,    11,   290,   484,   547,   845, 12661,
           553,   531,  1583,    13,  3271,   311,    13,   311, 28210,    11,
           257,  6240,   286, 45424,   379,   262,  2059,   286,  3442,    11,
         14727,    13,   366,  2990,   547,   845, 12661,    11,   290,   484,
           547,   845, 12661,    11,   290,   484,   547,   845, 12661,    11,
           290,   484,   547,   845, 12661,    11,   290,   484,   547,   845,
         12661,    11,   290,   484,   547,   845, 12661,    11,   290,   484,
           547,   845, 12661,    11,   290,   484,  

그리드 서치 디코딩의 단점 
- 그리드 서치 알고리즘은 반복적인 출력 시퀀스를 새성하는 경향이 있어 뉴스 기사로는 적절치 않다. 이는 그리드 서치 알고리즘의 보편적인 문제이다.
- 디코딩 측면에서 보면, 확률이 높은 단어가 확률이 낮은 단어보다 먼저 등장하기에 전체적으로 확률이 높은 단어 시퀀스를 생성하지 못하기도 한다.

그리디 서치 디코딩은 다양성이 필요한 텍스트 생성 작업에는 거의 사용되지 않지만, 결정적이고 사실적으로 정확한 출력이 필요한 수식 등의 짧은 문장 생성에는 유용하다.

# 빔 서치 디코딩
각 스텝에서 확률이 가장 높은 토큰을 선택하는 대신, 확률이 가장 높은 상위 b개의 다음 토큰을 추적한다. 다음 빔 세트는 기존 세트에서 가능한 모든 다음 토큰을 확장하고 확률이 가장 높은 b개의 확장을 선택해 구성한다. 이 과정은 최대 길이나 EOS토큰에 도달할 때까지 반복한다. 확률이 가장 높은 시퀀스는 로그확률에 따라 b개 빔의 순위를 매겨 선택된다.

트랜스포머스 모델은 입력 토큰이 주어지면 다음 토큰에 대한 정규화되지 않은 로짓을 반환한다. 따라서 먼저 로짓을 정규화해서 시퀀스의 각 토큰을 위해 전체 어휘사전에 대한 확률 분포를 만든다. 그 다음 시퀀스에 있는 토큰 확률만 선택한다. 아래 함수는 이 단계를 구현한 함수이다.

In [29]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
  logp = F.log_softmax(logits, dim = -1) #log(softmax(x)) 로짓 넣으면 로그확률 나온다.
  logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1) #로그확률
  return logp_label

이 함수는 하나의 토큰에 대한 로그 확률을 제공하므로, 시퀀스의 전체 로그 확률을 얻으려면 각 토큰의 로그 확률을 더한다.

In [30]:
def sequence_logprob(model, labels, input_len = 0):
  with torch.no_grad():
    output = model(labels)
    log_probs = log_probs_from_logits( #로짓으로부터 로그확률을 도출
        output.logits[:, :-1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:]) #로그확률 전부 더하기
    return seq_log_prob.cpu().numpy()

모델이 입력 시퀀스를 생성하지 않았으므로 입력 시퀀스의 로그 확률은 무시한다. 또한 로짓과 레이블의 정렬이 중요하다.  
모델은 다음 토큰을 예측하기에 첫 번째 레이블에 대한 로짓을 얻지 못한다. 또 마지막 로짓에 대한 정답이 없기에 마지막 로짓은 필요하지 않는다.

이 함수를 통해 OpenAI 프롬프트에서 그리디 서치로 만든 시퀀스의 로그 확률을 계산해보자.

In [31]:
logp = sequence_logprob(model, output_greedy, input_len = len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\n 로그 확률 : {logp : .2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


"The unicorns were very intelligent, and they were very intelligent," said Dr.
David S. Siegel, a professor of anthropology at the University of California,
Berkeley. "They were very intelligent, and they were very intelligent, and they
were very intelligent, and they were very intelligent, and they were very
intelligent, and they were very intelligent, and they were very intelligent, and
they were very

 로그 확률 : -83.33


이를 빔 서치로 생성한 시퀀스와 비교해보자.  

generate() 함수에서 빔 서치를 활성화하려면 num_beams 매개변쉥 빔 개수를 지정한다. 빔 크기가 클수록 결과가 더 좋을 가능성이 높다. 하지만 각 빔에 대해 병렬적으로 시퀀스를 생서하므로 생성 과정이 훨씬 느리다.

In [32]:
output_beam = model.generate(input_ids, max_length = max_length, num_beams = 5, do_sample = False)
logp = sequence_logprob(model, output_beam, input_len = len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n 로그 확률 : {logp:.2f}")

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


The researchers, from the University of California, San Diego, and the
University of California, Santa Cruz, found that the unicorns were able to
communicate with each other in a way that was similar to that of human speech.


"The unicorns were able to communicate with each other in a way that was similar
to that of human speech," said study co-lead author Dr. David J.

 로그 확률 : -78.34


단순한 그리디 서치보다 빔 서치가 더 높은 로그 확률을 얻었다. 하지만 빔 서치도 텍스트가 반복되는 문제가 있다. 이를 해결하기 위해 no_repeat_ngram_size 매개변수로 n-그램 페널티를 부과하는 방법이 있다. 그러면 지금까지 n-그램을 추적해 이전에 보았던 n-그램을 생성하는 경우 다음 토큰 확률이 0이 된다.

In [33]:
#generate : 문장 생성 함수
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n로그 확률: {logp:.2f}")

#점수는 더 낮아졌지만 텍스트가 일관성을 유지한다.

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


The researchers, from the University of California, San Diego, and the National
Science Foundation (NSF) in Boulder, Colorado, were able to translate the words
of the unicorn into English, which they then translated into Spanish.

"This is the first time that we have translated a language into an English
language," said study co-author and NSF professor of linguistics and
evolutionary biology Dr.

로그 확률: -101.88


분야에 국한되지 않는 잡담이나 기사처럼 사실적인 정확성이 다양성보다 덜 중요한 경우, 샘플링을 사용해 다양성을 늘리면서 반복을 줄이는 방법도 있다.  

많이 사용되는 샘플링 방법 몇 가지를 조사해보자.

# 샘플링 방법

온도가 생성되는 텍스트에 어떤 영향을 미치는지 알아보기 위해 generate() 함수의 temperature 매개변수를 T=2로 지정해 샘플링해보자.

In [35]:
#온도가 높으면 이상한 텍스트가 생성된다. 드문 토큰이 강조되어 모델이 이상한 문법을 만들고 다양한 가짜 단어를 만들어냈다.
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature = 2.0, top_k = 0) #max_length : 128
print(tokenizer.decode(output_temp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


Researchers tortured Clemson* four headed frail goats taken pur), Luigiimil
tsunefANDtg villains OFF admission glasses trb144 Symptoms Danger Thinkadders
large Tap tea --------- crou Cotton Chun desired rom Indeed 192 discontsers
beetles horses crops rook sizeof catsvery builtearWindow vs 2014PSTimer
applesandro((each 92 at eurosNE spectimensathered pegged002 46 Families common
h= 01adjusted N XenHQChristopherMax copies


In [37]:
output_temp = model.generate(input_ids, max_length = max_length, do_sample = True, temperature = 0.5, top_k = 0)
print(tokenizer.decode(output_temp[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a
remote, previously unexplored valley, in the Andes Mountains. Even more
surprising to the researchers was the fact that the unicorns spoke perfect
English.


"The unicorns speak only English, but they also speak a lot of Spanish and
Spanish-speaking languages," said study co-author Dr. Michael T. Brooks, a
professor of biology at the University of Colorado Boulder. "What we found was
that they had a unique language of speech that transcended languages of other
species."


The researchers found that the unicorns also had a unique way of


훨씬 일관성있는 텍스트가 나왔다.  

온도는 샘플의 품질을 제어하지만, 항상 일관성(낮은 온도)과 다양성(높은 온도)의 균형점이 있기에 당면한 문제에 따라 조정해야 한다.


일관성고 다양성의 균형을 조정하는 또 다른 방법으로 어휘사전의 분포를 잘라내는 방법이 있다. 이는 온도와 함께 다양성을 자유롭게 조정하지만, 더 제한된 범위에서 문맥상 매우 이상한 단어를 제외한다. 대표적으로 탑-k 샘플링과 뉴클리어스 샘플링이 있다.

# 탑-K 및 뉴클리어스 샘플링

탑-k와 뉴클리어스(탑-p) 샘플링은 온도를 사용하는 방법에 대한 잘 알려진 두 가지 대안 또는 확장이다. 두 샘플링은 모두 각 타임스텝에서 샘플링에 사용할 토큰의 개수를 줄인다.